# **ETL del archivo Homicidios_Hechos**

## **1.Cargando Las Librerías Necesarias**

In [1]:
import pandas as pd #Cargaremos pandas para manejar los datos de nuestros datasets,
import numpy as np #Cargaremos numpy para manejar los datos,
import gzip #Cargaremos gzip para poder abrir archivos con compresion gzip",
import datetime #Cargaremos datetime para manejar fechas",
from google.colab import drive #Cargaremos drive para poder acceder a los archivos de drive"
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
url01="/content/drive/MyDrive/2023/Henry/PI02-DataAnalyst/Datos/homicidios.xlsx"
url02="/content/drive/MyDrive/2023/Henry/PI02-DataAnalyst/Datos/homicidios_victimas.csv"

## **2.Fase ETL**

### 2.1.Carga de Datos

In [3]:
df_homicidios_victimas=pd.read_excel(url01,sheet_name="VICTIMAS")

In [4]:
df_homicidios_victimas.to_csv(url02,index=True,header=True,encoding='utf-8')

In [5]:
df_homicidios_victimas=pd.read_csv(url02,encoding='utf-8')

In [6]:
df_homicidios_victimas.head()

,Unnamed: 0,ID_hecho,FECHA,AAAA,MM,DD,ROL,VICTIMA,SEXO,EDAD,FECHA_FALLECIMIENTO
0,0,2016-0001,2016-01-01,2016,1,1,CONDUCTOR,MOTO,MASCULINO,19,2016-01-01 00:00:00
1,1,2016-0002,2016-01-02,2016,1,2,CONDUCTOR,AUTO,MASCULINO,70,2016-01-02 00:00:00
2,2,2016-0003,2016-01-03,2016,1,3,CONDUCTOR,MOTO,MASCULINO,30,2016-01-03 00:00:00
3,3,2016-0004,2016-01-10,2016,1,10,CONDUCTOR,MOTO,MASCULINO,18,SD
4,4,2016-0005,2016-01-21,2016,1,21,CONDUCTOR,MOTO,MASCULINO,29,2016-02-01 00:00:00


### 2.2.Analisis Descriptivo De Los Datos

#### 2.2.1.Funciones para el analisis descriptivo de los datos

In [7]:
def check_df(dataset, head = 5):

  """
  Funcion creada para tener una vista general de la base de datos
  """

  print('*'*30 + 'Forma de la base de datos' + '*'*30, end = '\n'*2)
  print(dataset.shape, end = '\n'*2)#Tamaño de la base de datos

  print('*'*30 + 'Informacion general de la base de datos' + '*'*30, end = '\n'*2)
  print(dataset.info(), end = '\n'*2)# Informacion general

  print('*'*30 + 'NaN cantidad de nulos' + '*'*30, end = '\n'*2)
  print(dataset.isnull().sum(), end = '\n'*2)#Cuenta la cantidad de NaN por columna

#### 2.2.2.Descripcion De Los Datos

In [8]:
check_df(df_homicidios_victimas)

******************************Forma de la base de datos******************************

(717, 11)

******************************Informacion general de la base de datos******************************

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 717 entries, 0 to 716
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Unnamed: 0           717 non-null    int64 
 1   ID_hecho             717 non-null    object
 2   FECHA                717 non-null    object
 3   AAAA                 717 non-null    int64 
 4   MM                   717 non-null    int64 
 5   DD                   717 non-null    int64 
 6   ROL                  717 non-null    object
 7   VICTIMA              717 non-null    object
 8   SEXO                 717 non-null    object
 9   EDAD                 717 non-null    object
 10  FECHA_FALLECIMIENTO  717 non-null    object
dtypes: int64(4), object(7)
memory usage: 61.7+ KB
None

**

### 2.3.Transformacion De Los Datos

#### 2.3.1.Funciones para la transformacion de los datos

In [ ]:
def Transform_Hora(text):

  '''

  La funcion de Transform_Hora recibe como parametro un dato de cualquier tipo y
  devuelve un objeto de tipo datetime extrayendo la hora del dato.
  Observando 2 formatos de fecha y un tipo de dato string entonces procedemos de
  la siguiente manera :
  format_full='%Y-%m-%d %H:%M:%S'
  format_short='%H:%M:%S'
  definiremos el formato que se presentan y se extraera posteriormente

  '''
  text=str(text)
  format_full='%Y-%m-%d %H:%M:%S'
  format_short='%H:%M:%S'
  if text.count("-")>=2:
    text.replace("1900","2018")
    datetime_object = datetime.datetime.strptime(text, format_full).time()

  elif text.count(":")>=1:
    datetime_object = datetime.datetime.strptime(text, format_short).time()

  else:
    datetime_object="SD"
  return  datetime_object


def Transform_Dir(text:str):
  """

  Esta funcion esta diseñada para reordenar la columna LUGAR_DEL_HECHO teniendo
  como principla estructura teniendo como prefijo AV. y DR. y eliminando la coma
  normaliando las abreviaciones en general

  """
  text=str(text).upper()
  if "," in text:
    parts = [part.strip() for part in text.split(",")]
    if len(parts) == 2:
        text= f"{parts[1]} {parts[0]} "
    if len(parts) == 3:
        text= f"{parts[2]} {parts[1]} {parts[0]} "
  if "DR." in text:
    parts = [part.strip() for part in text.split("DR.")]

    if len(parts) == 2:
      text= f"DR. {parts[0]} {parts[1]}"

  if "AV." in text:
    parts = [part.strip() for part in text.split("AV.")]

    if len(parts) == 2:
      text= f"AV. {parts[0]} {parts[1]}"



  else:
    text=""

  text=text.replace('GRAL.','GENERAL').replace('TTE.','TENIENTE').replace('CNEL.','CORONEL')
  text=text.strip()

  return text

#### 2.3.2.Transformacion de los datos

In [9]:
df_homicidios_victimas.columns

Index(['Unnamed: 0', 'ID_hecho', 'FECHA', 'AAAA', 'MM', 'DD', 'ROL', 'VICTIMA',
       'SEXO', 'EDAD', 'FECHA_FALLECIMIENTO'],
      dtype='object')

Se reviso las primeras las primeras 5  columnas **'ID', 'N_VICTIMAS', 'FECHA', 'AAAA', 'MM', 'DD'** y no muestran errores como tal

In [13]:
df_homicidios_victimas['FECHA'] = pd.to_datetime(df_homicidios_victimas['FECHA'])

In [18]:
df_homicidios_victimas['ROL'].value_counts()

CONDUCTOR               330
PEATON                  267
PASAJERO_ACOMPAÑANTE     80
CICLISTA                 29
SD                       11
Name: ROL, dtype: int64

In [23]:
df_homicidios_victimas[df_homicidios_victimas['ROL']=='SD']

,Unnamed: 0,ID_hecho,FECHA,AAAA,MM,DD,ROL,VICTIMA,SEXO,EDAD,FECHA_FALLECIMIENTO
36,36,2016-0049,2016-04-17,2016,4,17,SD,SD,SD,SD,SD
39,39,2016-0052,2016-04-20,2016,4,20,SD,MOTO,SD,SD,SD
63,63,2016-0085,2016-06-29,2016,6,29,SD,MOTO,MASCULINO,SD,SD
77,77,2016-0101,2016-08-07,2016,8,7,SD,SD,MASCULINO,67,SD
89,89,2016-0115,2016-09-02,2016,9,2,SD,SD,MASCULINO,SD,SD
141,141,2016-0174,2016-12-27,2016,12,27,SD,SD,SD,SD,SD
167,167,2017-0029,2017-03-07,2017,3,7,SD,SD,MASCULINO,34,2017-03-07 00:00:00
208,208,2017-0074,2017-06-04,2017,6,4,SD,SD,MASCULINO,70,2017-06-04 00:00:00
221,221,2017-0089,2017-07-13,2017,7,13,SD,SD,MASCULINO,23,SD
280,280,2017-0155,2017-12-12,2017,12,12,SD,SD,MASCULINO,77,SD


In [21]:
df_homicidios_victimas['VICTIMA'].value_counts()

MOTO         303
PEATON       267
AUTO          94
BICICLETA     29
SD             9
CARGAS         7
PASAJEROS      5
MOVIL          3
Name: VICTIMA, dtype: int64

In [22]:
df_homicidios_victimas[df_homicidios_victimas['VICTIMA']=='SD']

,Unnamed: 0,ID_hecho,FECHA,AAAA,MM,DD,ROL,VICTIMA,SEXO,EDAD,FECHA_FALLECIMIENTO
36,36,2016-0049,2016-04-17,2016,4,17,SD,SD,SD,SD,SD
77,77,2016-0101,2016-08-07,2016,8,7,SD,SD,MASCULINO,67,SD
89,89,2016-0115,2016-09-02,2016,9,2,SD,SD,MASCULINO,SD,SD
93,93,2016-0119,2016-09-04,2016,9,4,PASAJERO_ACOMPAÑANTE,SD,FEMENINO,SD,SD
141,141,2016-0174,2016-12-27,2016,12,27,SD,SD,SD,SD,SD
167,167,2017-0029,2017-03-07,2017,3,7,SD,SD,MASCULINO,34,2017-03-07 00:00:00
208,208,2017-0074,2017-06-04,2017,6,4,SD,SD,MASCULINO,70,2017-06-04 00:00:00
221,221,2017-0089,2017-07-13,2017,7,13,SD,SD,MASCULINO,23,SD
280,280,2017-0155,2017-12-12,2017,12,12,SD,SD,MASCULINO,77,SD


In [24]:
df_homicidios_victimas['SEXO'].value_counts()

MASCULINO    545
FEMENINO     166
SD             6
Name: SEXO, dtype: int64

In [25]:
df_homicidios_victimas[df_homicidios_victimas['SEXO']=='SD']

,Unnamed: 0,ID_hecho,FECHA,AAAA,MM,DD,ROL,VICTIMA,SEXO,EDAD,FECHA_FALLECIMIENTO
36,36,2016-0049,2016-04-17,2016,4,17,SD,SD,SD,SD,SD
39,39,2016-0052,2016-04-20,2016,4,20,SD,MOTO,SD,SD,SD
108,108,2016-0136,2016-10-25,2016,10,25,CONDUCTOR,MOTO,SD,SD,SD
121,121,2016-0151,2016-11-18,2016,11,18,PEATON,PEATON,SD,SD,SD
138,138,2016-0171,2016-12-25,2016,12,25,CONDUCTOR,MOTO,SD,SD,SD
141,141,2016-0174,2016-12-27,2016,12,27,SD,SD,SD,SD,SD


In [26]:
df_homicidios_victimas['EDAD'].value_counts()

SD    53
30    28
29    24
23    24
27    20
      ..
11     1
13     1
85     1
7      1
88     1
Name: EDAD, Length: 86, dtype: int64

In [27]:
df_homicidios_victimas[df_homicidios_victimas['EDAD']=='SD']

,Unnamed: 0,ID_hecho,FECHA,AAAA,MM,DD,ROL,VICTIMA,SEXO,EDAD,FECHA_FALLECIMIENTO
30,30,2016-0041,2016-03-29,2016,3,29,PASAJERO_ACOMPAÑANTE,MOTO,MASCULINO,SD,2016-03-30 00:00:00
33,33,2016-0045,2016-04-11,2016,4,11,CONDUCTOR,MOTO,MASCULINO,SD,SD
35,35,2016-0048,2016-04-15,2016,4,15,PEATON,PEATON,FEMENINO,SD,SD
36,36,2016-0049,2016-04-17,2016,4,17,SD,SD,SD,SD,SD
39,39,2016-0052,2016-04-20,2016,4,20,SD,MOTO,SD,SD,SD
55,55,2016-0077,2016-06-13,2016,6,13,PEATON,PEATON,FEMENINO,SD,SD
63,63,2016-0085,2016-06-29,2016,6,29,SD,MOTO,MASCULINO,SD,SD
72,72,2016-0096,2016-07-25,2016,7,25,CONDUCTOR,MOTO,MASCULINO,SD,SD
89,89,2016-0115,2016-09-02,2016,9,2,SD,SD,MASCULINO,SD,SD
93,93,2016-0119,2016-09-04,2016,9,4,PASAJERO_ACOMPAÑANTE,SD,FEMENINO,SD,SD


In [28]:
df_homicidios_victimas['FECHA_FALLECIMIENTO'].value_counts()

SD                     68
2017-02-26 00:00:00     3
2017-01-16 00:00:00     3
2020-12-25 00:00:00     3
2019-12-18 00:00:00     3
                       ..
2018-01-19 00:00:00     1
2018-01-18 00:00:00     1
2018-01-14 00:00:00     1
2018-01-12 00:00:00     1
2022-01-02 00:00:00     1
Name: FECHA_FALLECIMIENTO, Length: 563, dtype: int64

In [29]:
df_homicidios_victimas[df_homicidios_victimas['FECHA_FALLECIMIENTO']=='SD']

,Unnamed: 0,ID_hecho,FECHA,AAAA,MM,DD,ROL,VICTIMA,SEXO,EDAD,FECHA_FALLECIMIENTO
3,3,2016-0004,2016-01-10,2016,1,10,CONDUCTOR,MOTO,MASCULINO,18,SD
16,16,2016-0022,2016-02-21,2016,2,21,PASAJERO_ACOMPAÑANTE,MOTO,MASCULINO,41,SD
19,19,2016-0027,2016-02-28,2016,2,28,PASAJERO_ACOMPAÑANTE,AUTO,MASCULINO,34,SD
22,22,2016-0031,2016-03-08,2016,3,8,CONDUCTOR,MOTO,MASCULINO,21,SD
33,33,2016-0045,2016-04-11,2016,4,11,CONDUCTOR,MOTO,MASCULINO,SD,SD
...,...,...,...,...,...,...,...,...,...,...,...
221,221,2017-0089,2017-07-13,2017,7,13,SD,SD,MASCULINO,23,SD
242,242,2017-0112,2017-09-10,2017,9,10,PASAJERO_ACOMPAÑANTE,AUTO,MASCULINO,1,SD
246,246,2017-0115,2017-09-19,2017,9,19,CONDUCTOR,MOTO,MASCULINO,34,SD
253,253,2017-0126,2017-10-14,2017,10,14,PASAJERO_ACOMPAÑANTE,AUTO,MASCULINO,39,SD


In [30]:
df_homicidios_victimas['FECHA_FALLECIMIENTO'].unique()

array(['2016-01-01 00:00:00', '2016-01-02 00:00:00',
       '2016-01-03 00:00:00', 'SD', '2016-02-01 00:00:00',
       '2016-01-24 00:00:00', '2016-01-26 00:00:00',
       '2016-01-29 00:00:00', '2016-02-08 00:00:00',
       '2016-02-10 00:00:00', '2016-02-14 00:00:00',
       '2016-02-16 00:00:00', '2016-02-17 00:00:00',
       '2016-03-02 00:00:00', '2016-02-28 00:00:00',
       '2016-03-04 00:00:00', '2016-03-12 00:00:00',
       '2016-03-13 00:00:00', '2016-03-14 00:00:00',
       '2016-03-19 00:00:00', '2016-03-21 00:00:00',
       '2016-03-23 00:00:00', '2016-03-29 00:00:00',
       '2016-03-30 00:00:00', '2016-03-31 00:00:00',
       '2016-04-15 00:00:00', '2016-04-19 00:00:00',
       '2016-04-20 00:00:00', '2016-04-22 00:00:00',
       '2016-04-25 00:00:00', '2016-04-28 00:00:00',
       '2016-04-26 00:00:00', '2016-05-08 00:00:00',
       '2016-05-26 00:00:00', '2016-05-20 00:00:00',
       '2016-05-23 00:00:00', '2016-06-12 00:00:00',
       '2016-06-13 00:00:00', '2016-07-0